# PRÁCTICA 1

Lucía Pérez González, Manuel Ramallo Blanco, Alexandre Lorenzo Martínez

## 1 - Preprocesado

### 1.1 - Eliminación de duplicados

In [1]:
# Abrir datasets
import pandas as pd

df_vino = pd.read_csv("data/train.csv")

# Eliminación de duplicados, ignorado quality
cols = df_vino.columns.drop('quality')
df_vino = df_vino.drop_duplicates(subset=cols)

### 1.2 - Binarización de la calidad

In [2]:
def clasificar_vino(valor): 
    if valor < 7: 
        return "baja calidad" 
    elif valor >= 7: 
        return "alta calidad" 
df_vino['calidad'] = df_vino['quality'].apply(clasificar_vino) 
df_vino = df_vino.drop(columns=['quality'])

### 1.3 - Gestión de valores atípicos

Eliminamos aquellas filas que no tienen valores para 4 columnas o más:

In [6]:
df_vino = df_vino[df_vino.isnull().sum(axis=1) < 4]

Aquellas filas que tengan 4 o más valores atípicos también serán descartadas. Para contar cuántos datos atípicos hay por fila, añadimos una nueva columna.

In [7]:
df_vino['num atipicos'] = 0

C:\Users\User\AppData\Local\Temp\ipykernel_20336\3018258264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vino['num atipicos'] = 0


#### 1.3.1 - Fixed acidity

#### 1.3.2 - Volatile acidity

#### 1.3.3 - Citric acid

#### 1.3.4 - Residual sugar

#### 1.3.5 - Chlorides

#### 1.3.6 - Free sulfur dioxide

#### 1.3.7 - Total sulfur dioxide

#### 1.3.8 - Density

#### 1.3.8 - pH

#### 1.3.9 - Sulphates

#### 1.3.10 - Alcohol

#### 1.3.11 - Eliminación de filas con muchos datos atípicos

In [8]:
df_vino = df_vino[df_vino['num atipicos'] < 4]

In [15]:
# Normalización de los datos
from sklearn.preprocessing import StandardScaler

def estandarizar_train_test(train_df, test_df, target):
    columnas = train_df.drop(columns=[target]).select_dtypes(include="number").columns

    scaler = StandardScaler()
    train_df_scaled = train_df.copy()
    test_df_scaled = test_df.copy()

    train_df_scaled[columnas] = scaler.fit_transform(train_df[columnas])
    test_df_scaled[columnas] = scaler.transform(test_df[columnas])

    return train_df_scaled, test_df_scaled


In [13]:
# Selección de características con SelectKBest y correlación
from sklearn.feature_selection import SelectKBest, f_regression

def seleccion_kbest(df, k, target='calidad'):
    X = df.drop(columns=[target])
    y = df[target]

    selector = SelectKBest(score_func=f_regression, k=k)
    selector.fit(X, y)

    selected_features = X.columns[selector.get_support()]

    df_fs = df[selected_features.tolist() + [target]]
    return df_fs

def seleccion_correlacion(df, n, target='calidad'):
    corr = df.corr(numeric_only=True)[target].abs().sort_values(ascending=False)
    selected_features = corr.index[1:n+1]
    return df[selected_features.tolist() + [target]]

def seleccion_caracteristicas(df, n, funcion, target='calidad'):
    if funcion == 'kbest':
        return seleccion_kbest(df, n, target)
    elif funcion == 'correlacion':
        return seleccion_correlacion(df, n, target)
    else:
        raise ValueError("Función de selección no válida. Use 'kbest' o 'correlacion'.")


In [4]:
def preprocesado(df_train, df_valid, target='calidad'):
    return 0, 0

In [ ]:
# Modelo de entrenamiento
import numpy as np
import pandas as pd

def nested_cv(df, target='calidad'):
    """
    Validación cruzada anidada:
    - exterior: 3 iteraciones (6 pedazos, 4 train, 2 val)
    - interior: 4 pedazos (3 train, 1 val)
    
    Devuelve: diccionario con información de folds
    """
    
    # Mezclar el dataset para aleatoriedad
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = len(df_shuffled)
    
    # Dividir en 6 pedazos iguales (outer)
    outer_splits = np.array_split(df_shuffled, 6)
    
    results = []
    
    # Outer CV
    for outer_iter in range(3):
        # Elegimos 4 pedazos para train, 2 para val
        outer_train_idx = [(outer_iter + i) % 6 for i in range(4)]
        outer_valid_idx = [(outer_iter + 4 + i) % 6 for i in range(2)]
        
        train_outer = pd.concat([outer_splits[i] for i in outer_train_idx])
        valid_outer = pd.concat([outer_splits[i] for i in outer_valid_idx])
        
        # Dividir train_outer en 4 pedazos para inner CV
        inner_splits = np.array_split(train_outer.sample(frac=1, random_state=outer_iter), 4)
        
        inner_results = []
        
        # Inner CV
        for inner_iter in range(4):
            inner_valid_pre = inner_splits[inner_iter]
            inner_train_pre = pd.concat([s for j, s in enumerate(inner_splits) if j != inner_iter])
            
            # Preprocesar
            df_train, df_val = preprocesado(inner_train_pre, inner_valid_pre, target)
            
            # --- Aquí va el entrenamiento de los modelos ---

            # Guardar info del cada ejecución del fold interno
            inner_results.append({
                "inner_iter": inner_iter,
                "train_rows": len(df_train),
                "valid_rows": len(df_val),
                # "score": score
            })
        
        # Preprocesar outer validation usando train outer
        train_outer_proc, valid_outer_proc = preprocesado(train_outer, valid_outer, target)
        
        # aqui entreno en train_outer_proc el modelo con mejor puntuación del la VC interna y evalúo en valid_outer_proc
        

        results.append({
            "outer_iter": outer_iter,
            "train_rows_outer": len(train_outer_proc),
            "valid_rows_outer": len(valid_outer_proc),
            "inner_folds": inner_results,
            # "outer_score": outer_score
        })

    return results
